In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair
import data.scripts.gen_tables.search_best_inter as search_inter

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 
CSV_PATH = os.path.join(HOME, 'data/csv') 
GRAPH_PATH = os.path.join(HOME, 'data/graphs')

%config InlineBackend.figure_format ='retina'
plt.style.use('seaborn-talk')

In [3]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [4]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
seq_file = os.path.join(CSV_PATH, 'seq-multi.csv')
seq_pkl = os.path.join(PKL_PATH, 'seq-multi.pkl')
%run $gen_seq --multi --csv $seq_file --output $seq_pkl

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq-multi.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'l2_parallelism', 'avg_mem_lat', 'avg_core_to_l2',
       'avg_l2_to_core', 'avg_mrq_latency', 'stall_core_ldst',
       'stall_icnt_to_l2', 'l2_BW', 'l2_rshr_entry_fail', 'l2_rshr_merge_fail',
       'stall_l2_to_icnt', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_total_accesses', 'packet_lat_out', 'network_lat_out', 'inject_out',
       'accepted_out', 'packet_lat_in', 'network_lat_in', 'inject_in',
       'accepted_in', 'mem_subpartition_parallism',
       'mem_subpartition_parallism_util', 'L2_reservation_fail', 'empty_warp',
       'stall_warp', 'idle_warp', 'scoreboard_warp', 'tot_warp_insn', 'regs',
       'smem', 'mem_count', 'sp_winsn', 'dp_winsn', 'int_winsn',
       'tensor_winsn', 'sfu_winsn', 'mem_insn', 'sp_busy', 'dp_busy',
       'int_busy', 'tensor_busy', 'sfu_busy', 'barrier_cycles',
       'in

In [5]:
col_seq = ['pair_str', 'kidx', 'runtime',
           'ipc',
           'avg_dram_bw', 
           'sp_busy', 'dp_busy', 'int_busy', 'tensor_busy', 'sfu_busy',
          ]

draw_table(df_seq, col_seq).format({
    'runtime': '{:,}',
    'avg_dram_bw':'{:.4f}', 
    'sp_busy': '{:.2f}',
    'dp_busy': '{:.2f}',
    'int_busy': '{:.2f}',
    'tensor_busy': '{:.2f}',
    'sfu_busy': '{:.2f}',
})

pair_str,kidx,runtime,ipc,avg_dram_bw,sp_busy,dp_busy,int_busy,tensor_busy,sfu_busy
nvd_conv-0,1,"190,346",2131.88,0.7242,0.08,0.00,0.02,0.00,0.00
nvd_conv-0,2,"208,164",1755.9,0.6698,0.08,0.00,0.03,0.00,0.00
parb_sad-0,1,"51,633",2686.34,0.0061,0.01,0.00,0.23,0.00,0.00
parb_sad-0,2,"21,556",203.18,0.4558,0.00,0.00,0.01,0.00,0.00
parb_sad-0,3,"12,806",89.8946,0.1249,0.00,0.00,0.01,0.00,0.00
parb_sad-1,1,"3,000,706",3754.07,0.1736,0.02,0.00,0.32,0.00,0.00
parb_sad-1,2,"2,324,970",153,0.5645,0.00,0.00,0.01,0.00,0.00
parb_sad-1,3,"541,865",172.561,0.5842,0.00,0.00,0.01,0.00,0.00


# Intra

In [8]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
intra_file = os.path.join(CSV_PATH, 'intra-multi.csv')
intra_pkl = os.path.join(PKL_PATH, 'intra-multi.pkl')
%run $gen_intra --out_intra $intra_pkl --seq $seq_pkl --csv $intra_file

In [13]:
col_prod = ['norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 
           'sum_comp', 'sum_dram', 'penalized']
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')
pairs = df_seq.apply(lambda row: ':'.join([row['pair_str'], str(row['kidx'])]), axis=1)

In [14]:
app1 = widgets.Dropdown(
    options=pairs,
    value=pairs[0],
    rows=5,
    description='App 1:',
    disabled=False
)

app2 = widgets.Dropdown(
    options=pairs,
    value=pairs[0],
    rows=5,
    description='App 2:',
    disabled=False
)

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    
show_widgets()

def onclick(b):
    clear_output()
    show_widgets()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value --intra_pkl $intra_pkl
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
    print(df_prod.columns)
#     sns.lineplot('diff_mflat', 'sum_ipc', data=df_prod)
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    
button.on_click(onclick)

Dropdown(description='App 1:', index=2, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2…

Dropdown(description='App 2:', options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_sad-0:1', 'parb_sad-0:2', 'parb_…

FloatSlider(value=0.5, continuous_update=False, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

Error. No feasible pair configs for parb_sad-0:1+nvd_conv-0:1 at QoS 0.5.
Index(['index', 'pair_str_x', 'config_x', 'gpusim_version_x', 'jobId_x',
       'grid_x_x', 'grid_y_x', 'grid_z_x', 'block_x_x', 'block_y_x',
       ...
       'thread_ratio_y', 'smem_ratio_y', 'reg_ratio_y', 'usage_y',
       'diff_mflat', 'sum_ipc', 'sum_comp', 'sum_dram', 'penalized', 'config'],
      dtype='object', length=208)


,norm_ipc_x,norm_ipc_y,diff_mflat,sum_ipc,intra_x,intra_y,sum_comp,sum_dram,penalized
0,0.817747,1.04297,24,1.86072,4,4,0.426986,0.940433,parb_sad-0
1,0.999458,1.16624,29,2.16569,8,8,0.532725,0.957803,nvd_conv-0
2,0.817747,1.01166,39,1.8294,4,4,0.462074,0.916257,parb_sad-0
3,0.981934,1.04082,82,2.02275,16,12,0.563441,0.955677,nvd_conv-0
4,1,1.04082,102,2.04082,17,12,0.585803,0.956408,nvd_conv-0
5,0.981934,0.997046,129,1.97898,16,8,0.607487,0.915981,nvd_conv-0
6,1,0.997046,149,1.99705,17,8,0.629848,0.916712,nvd_conv-0
7,1.05067,0.997046,164,2.04771,12,8,0.598035,0.917809,parb_sad-0
8,1.05067,1.16624,173,2.2169,12,8,0.577752,0.957418,nvd_conv-0
9,1.05067,1.04082,211,2.09149,12,12,0.55399,0.957506,parb_sad-0
